# Test di Pearson

In [2]:
import os
import math
from pandas import *
import pandas as pd
from datetime import datetime
%matplotlib inline 
import matplotlib.pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF

os.getcwd()
os.chdir("/Users/riccardo/Dropbox/A-Data/Paperopoli")
data = pd.read_csv('PluviometriaPaperopoli_4.txt')
map(datetime,data['anno'])
data=data.set_index('anno')

In [3]:
data[-5:]

,1h,3h,6h,12h,24h
anno,,,,,
1984,61.2,61.4,61.4,62.6,81.8
1985,23.6,35.6,44.2,58.6,69.8
1986,20.2,36.8,51.4,55.8,74.4
1987,32.6,40.6,64.6,77.2,81.2
1988,89.2,102.0,102.0,102.0,104.2


In [4]:
from scipy.stats import norm
from scipy.stats import genextreme,gumbel_r
from numpy import linspace
from pylab import plot,show,hist,figure,title

In [5]:
os.listdir()

['abs_mle_right',
 'abs_mle',
 'abs_mq',
 'PluviometriaPaperopoli_4.txt',
 'best_param',
 'abs_m',
 'PluviometriaPaperopoli_2.txt',
 'PluviometriaPaperopoli_3.txt']

In [5]:
abs_m=pd.read_pickle("abs_m")
abs_mq=pd.read_pickle("abs_mq")
abs_mle=pd.read_pickle("abs_mle")

In [6]:
abs_m-abs_mq

,1h,3h,6h,12h,24h
a,-0.499291,-0.989925,-0.408223,-0.968343,2.212177
b,1.819930,-0.764857,-3.666225,-6.727317,0.300262


In [7]:
abs_m

,1h,3h,6h,12h,24h
a,31.241689,39.187045,48.028038,60.753736,75.683743
b,9.823556,10.694365,13.272078,18.313274,23.986088


In [8]:
abs_mq

,1h,3h,6h,12h,24h
a,31.740980,40.176970,48.436261,61.722079,73.471566
b,8.003627,11.459221,16.938302,25.040592,23.685827


In [9]:
abs_mle

,1h,3h,6h,12h,24h
a,31.59042,39.527315,48.100316,60.562103,75.848891
b,9.07316,10.041456,12.398915,18.005471,22.435148


https://docs.scipy.org/doc/scipy-0.18.1/reference/tutorial/stats.html

In [11]:
abs_m.at["a","1h"]

31.241689405642404

In [12]:
rv=gumbel_r(loc=abs_m.at["a","1h"],scale=abs_m.at["b","1h"])
rv

In [13]:
q=[0.2,0.4,0.6,0.8,1]
h1h=rv.ppf(q)
h1h

array([26.56680635, 32.10048014, 37.84043735, 45.97643435,         inf])

In [14]:
ecdf1h = ECDF(data["1h"])
r=ecdf1h(h1h)
r

array([0.16, 0.34, 0.62, 0.84, 1.  ])

In [15]:
l1h=len(data["1h"])
l1h

50

Ci si aspetta che, in ogni intervallo, con limite destro h1h, ci siano $e=(l1h * q)$ elementi mentre ce ne sono  in realtà $o=(l1h * r)$.  La formula dell'$X^2$ è allora:
\begin{equation}
X^2 = \frac{1}{e}\sum_{i=1}^k (e - o)^2
\end{equation}

In [16]:
o0=l1h*r
o0

array([ 8., 17., 31., 42., 50.])

In [17]:
np.append([0],np.delete(o0,-1))

array([ 0.,  8., 17., 31., 42.])

In [18]:
o=o0-np.append([0],np.delete(o0,-1))
o

array([ 8.,  9., 14., 11.,  8.])

In [19]:
range(len(r))

range(0, 5)

In [20]:
e=[0.2*l1h for i in range(len(r))]
e

[10.0, 10.0, 10.0, 10.0, 10.0]

In [21]:
o-e

array([-2., -1.,  4.,  1., -2.])

In [22]:
(o-e)**2

array([ 4.,  1., 16.,  1.,  4.])

In [23]:
(o-e)**2/e

array([0.4, 0.1, 1.6, 0.1, 0.4])

In [24]:
((o-e)**2/e).sum()

2.6000000000000045

In [25]:
def X2(data,abs_t,h,delta):    
    dt=data[h].dropna()
    #print(dt)
    lh=len(dt)
    #print("lh: ",lh)
    q=[delta*(i+1) for i in range(int(1/delta))]
    #print("q: ",q)
    rv=gumbel_r(loc=abs_t.at["a",h],scale=abs_t.at["b",h])
    ecdf = ECDF(dt)
    r=ecdf(rv.ppf(q))
    #print("r: ",r)
    o0=lh*r
    #print("o0: ",o0)
    o=o0-np.append([0],np.delete(o0,-1))
    #print("o: ",o)
    e=[delta*lh for i in range(len(r))]
    #print("e: ",e)
    return ((o-e)**2/e).sum()  

In [26]:
X2(data,abs_m,"1h",0.2)

2.6000000000000045

In [27]:
X2(data,abs_mq,"1h",0.2)

0.7999999999999978

In [28]:
X2(data,abs_mle,"1h",0.2)

1.8000000000000023

In [29]:
data.columns

Index(['1h', '3h', '6h', '12h', '24h'], dtype='object')

In [30]:
for  h in data.columns:
    print(h,": ",[X2(data,abs_m,h,0.2),X2(data,abs_mq,h,0.2),X2(data,abs_mle,h,0.2)])

1h :  [2.6000000000000045, 0.7999999999999978, 1.8000000000000023]
3h :  [2.2222222222222223, 6.222222222222225, 2.6666666666666665]
6h :  [4.222222222222215, 7.1111111111111125, 3.333333333333326]
12h :  [3.1111111111111085, 2.888888888888889, 3.7777777777777697]
24h :  [0.6666666666666667, 0.8888888888888888, 1.1111111111111112]


In [ ]:
abs_mle

In [ ]:
a=[]
for  h in data.columns:
    a=a+[[X2(data,abs_m,h,0.2),X2(data,abs_mq,h,0.2),X2(data,abs_mle,h,0.2)]]
   
pd.DataFrame(a,index=["1h","3h","6h","12h","24h"],columns=["M","Mq","MLE"])

La strategia è di costruire un DataFrame dei parametri migliori. 

In [ ]:
best=DataFrame([abs_mq["1h"],
                abs_m["3h"],
                abs_mle["6h"],
                abs_mq["12h"],
                abs_m["24h"]]).T
                  
best

In [ ]:
best.to_pickle("best_param")

In [ ]:
from scipy.stats import genextreme,gumbel_r
from numpy import linspace
from statsmodels.distributions.empirical_distribution import ECDF
def gumbel(x,abs_d,col):
    return np.exp(-np.exp(-(x-abs_d[col]["a"])/abs_d[col]["b"]))
t_rain=np.linspace(data.min()[0],data.max()[4],100)

In [ ]:
gb=pd.DataFrame([gumbel(t_rain,best,"1h"),
                 gumbel(t_rain,best,"3h"),
                 gumbel(t_rain,best,"6h"),
                 gumbel(t_rain,best,"12h"),
                 gumbel(t_rain,best,"24h")]).T
gb.index=t_rain
gb.columns=["1h","3h","6h","12h","24"]
gb[-5:]

In [ ]:
plt.rc('xtick', labelsize=20) #Quest  comandi globali
plt.rc('ytick', labelsize=20) #Questi sono comandi globali
ax=gb.plot(color=["red","blue","cyan","green","black"])
ecdf1h = ECDF(data["1h"])
ax.plot(data["1h"],ecdf1h(data["1h"]),'o',c="red")
data3h=data["3h"].dropna()
ecdf3h = ECDF(data3h)
ax.plot(data3h,ecdf3h(data3h),"o",c="blue")
data6h=data["6h"].dropna()
ecdf6h = ECDF(data6h)
ax.plot(data6h,ecdf6h(data6h),"o",c="cyan")
data12h=data["12h"].dropna()
ecdf12h = ECDF(data12h)
ax.plot(data12h,ecdf12h(data12h),"o",c="green")
data24h=data["24h"].dropna()
ecdf24h = ECDF(data24h)
ax.plot(data24h,ecdf24h(data24h),"o",c="black")
ax.set_title('Lssp 1h')
ax.set_xlabel('Rainfall (mm)')
ax.set_ylabel('P[H<h]')